# 와인 리뷰데이터 인덱싱
- 벡터 DB : pinecone

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

# Keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Pinecone index (create it on Pinecone console beforehand)
PINECONE_INDEX_NAME = "wine-review2"
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE") or "default"

# Pinecone integrated embedding model (matches your console setting)
PINECONE_EMBEDDING_MODEL = "llama-text-embed-v2"
PINECONE_EMBEDDING_DIM = 2048


## document loader

In [2]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
docs = loader.load()
for i, d in enumerate(docs[:3]):
    print(i, d)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

In [3]:
len(docs)

129971

## embedding 모델 객체 생성

In [4]:
# Embedding model (Pinecone Inference)
# - Index dimension: 2048 (wine-review2)
# - Model: llama-text-embed-v2
# This cell enforces:
#   1) dimension=2048 (must match the index)
#   2) max inputs per request <= 96 (model limit)
#   3) tokens-per-minute (TPM) throttling + 429 backoff

import time
from typing import List
from langchain_core.embeddings import Embeddings
from pinecone import Pinecone

PINECONE_EMBEDDING_MODEL = "llama-text-embed-v2"
PINECONE_EMBEDDING_DIM = 2048

# Model constraints / rate limits
MODEL_INPUTS_LIMIT = 96         # ✅ llama-text-embed-v2 inputs limit per request
TPM_LIMIT = 250_000             # ✅ from your error message (tokens per minute)

# Tuning knobs (adjust if you still see 429)
EMBED_BATCH_LIMIT = 8           # ✅ <= 96 (8~16 recommended; lower if chunks are long)
SAFETY = 1.35                   # ✅ safety margin for TPM throttling
MIN_SLEEP = 1.0                 # ✅ minimum pause per call (seconds)
MAX_RETRIES = 10                # ✅ retry count for 429
BACKOFF_BASE = 2.0              # ✅ exponential backoff multiplier

pc = Pinecone(api_key=PINECONE_API_KEY)

def estimate_tokens(text: str) -> int:
    """Rough token estimator.
    Works well enough for throttling: ~1 token per ~4 chars for English/mixed text.
    If your chunks are mostly Korean, you can make this stricter (e.g., //3).
    """
    return max(1, len(text) // 4)

class PineconeInferenceEmbeddings(Embeddings):
    def __init__(self, model: str, dim: int):
        self.model = model
        self.dim = dim

    def _chunk(self, arr: List[str], n: int):
        for i in range(0, len(arr), n):
            yield arr[i:i + n]

    def _sleep_for_tpm(self, inputs: List[str]):
        total_tokens = sum(estimate_tokens(t) for t in inputs)
        needed = (total_tokens / TPM_LIMIT) * 60.0 * SAFETY
        time.sleep(max(MIN_SLEEP, needed))

    def _embed_with_retry(self, inputs: List[str], input_type: str):
        if len(inputs) > MODEL_INPUTS_LIMIT:
            raise ValueError(f"Too many inputs for one embed call: {len(inputs)} > {MODEL_INPUTS_LIMIT}")

        delay = 1.0
        for _ in range(MAX_RETRIES):
            try:
                res = pc.inference.embed(
                    model=self.model,
                    inputs=inputs,
                    parameters={
                        "input_type": input_type,
                        "dimension": self.dim,  # ✅ enforce 2048 to match the index
                    },
                )
                self._sleep_for_tpm(inputs)
                return res
            except Exception as e:
                msg = str(e)
                if ("429" in msg) or ("RESOURCE_EXHAUSTED" in msg) or ("max tokens per minute" in msg):
                    time.sleep(delay)
                    delay *= BACKOFF_BASE
                    continue
                raise
        raise RuntimeError("Embedding rate limit: retries exceeded (429)")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        vectors: List[List[float]] = []
        for sub in self._chunk(texts, EMBED_BATCH_LIMIT):
            res = self._embed_with_retry(sub, input_type="passage")
            vectors.extend([x["values"] for x in res.data])
        return vectors

    def embed_query(self, text: str) -> List[float]:
        res = self._embed_with_retry([text], input_type="query")
        return res.data[0]["values"]

embeddings = PineconeInferenceEmbeddings(PINECONE_EMBEDDING_MODEL, PINECONE_EMBEDDING_DIM)

# Quick sanity check
print("embed dim =", len(embeddings.embed_query("dimension check")))  # should print 2048


embed dim = 2048


## Pinecone객체, index객체 생성

In [5]:
from pinecone import Pinecone, ServerlessSpec


# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [6]:
# pinecone에 index list 가져오기
existing_indexes = pc.list_indexes()

# 이름만 추출
index_names = [index['name'] for index in existing_indexes.indexes]
# print(index_names)


if PINECONE_INDEX_NAME not in index_names:
    pc.create_index(
        name=PINECONE_INDEX_NAME,
                dimension=PINECONE_EMBEDDING_DIM,  # must match embedding dimension
        metric="cosine",  # 모델 메트릭, openapi embeding model 에서 사용하는 것 확인
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{PINECONE_INDEX_NAME}' created successfully.")
else:
    print(f"Index '{PINECONE_INDEX_NAME}' already exists.")


Index 'wine-review2' already exists.


## Split 객체 생성

In [7]:
# split하기
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [8]:
len(chunks)

129982

In [9]:
chunks[2]

Document(metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 2}, page_content=': 2\ncountry: US\ndescription: Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.\ndesignation: \npoints: 87\nprice: 14.0\nprovince: Oregon\nregion_1: Willamette Valley\nregion_2: Willamette Valley\ntaster_name: Paul Gregutt\ntaster_twitter_handle: @paulgwine\ntitle: Rainstorm 2013 Pinot Gris (Willamette Valley)\nvariety: Pinot Gris\nwinery: Rainstorm')

## 배치 크기 단위로 저장하기
- langchain_pinecone의 PineconeVectorStore으로 벡터 DB에 저장

In [ ]:
# vector sotre에 저장(2시간정도 걸림)
from langchain_pinecone import PineconeVectorStore

BATCH_SIZE = 200  # 업서트 배치(문서가 길면 100~200 권장)  # 한 번에 처리할 문서 수(최대 vector 수 1000개, 2MB 이내)

for i in range(0, len(chunks), BATCH_SIZE):
    batch_docs = chunks[i:i+BATCH_SIZE]
    
    # 첫 번째 배치로 벡터 스토어 생성
    if i == 0:
        vector_store = PineconeVectorStore.from_documents(
            batch_docs,            # BATCH_SIZE 수 만큼의 chunk
            embedding=embeddings,  # 임베딩 벡터로 변환
            index_name=PINECONE_INDEX_NAME,   # index 이름
            namespace=PINECONE_NAMESPACE      
        )

    # 이후 배치는 생성한 벡터 스토어에 추가, # 내부적으로 임베딩 벡터로 변환
    else:
        vector_store.add_documents(batch_docs)    
    
    print(f"배치 {i//BATCH_SIZE + 1} 완료: {len(batch_docs)}개 문서 업로드")

배치 1 완료: 200개 문서 업로드
배치 2 완료: 200개 문서 업로드
배치 3 완료: 200개 문서 업로드
배치 4 완료: 200개 문서 업로드
배치 5 완료: 200개 문서 업로드
배치 6 완료: 200개 문서 업로드
배치 7 완료: 200개 문서 업로드
배치 8 완료: 200개 문서 업로드
배치 9 완료: 200개 문서 업로드
배치 10 완료: 200개 문서 업로드
배치 11 완료: 200개 문서 업로드
배치 12 완료: 200개 문서 업로드
배치 13 완료: 200개 문서 업로드
배치 14 완료: 200개 문서 업로드
배치 15 완료: 200개 문서 업로드
배치 16 완료: 200개 문서 업로드
배치 17 완료: 200개 문서 업로드
배치 18 완료: 200개 문서 업로드
배치 19 완료: 200개 문서 업로드
배치 20 완료: 200개 문서 업로드
배치 21 완료: 200개 문서 업로드
배치 22 완료: 200개 문서 업로드
배치 23 완료: 200개 문서 업로드
배치 24 완료: 200개 문서 업로드
배치 25 완료: 200개 문서 업로드
배치 26 완료: 200개 문서 업로드
배치 27 완료: 200개 문서 업로드
배치 28 완료: 200개 문서 업로드
배치 29 완료: 200개 문서 업로드
배치 30 완료: 200개 문서 업로드
배치 31 완료: 200개 문서 업로드
배치 32 완료: 200개 문서 업로드
배치 33 완료: 200개 문서 업로드
배치 34 완료: 200개 문서 업로드
배치 35 완료: 200개 문서 업로드
배치 36 완료: 200개 문서 업로드
배치 37 완료: 200개 문서 업로드
배치 38 완료: 200개 문서 업로드
배치 39 완료: 200개 문서 업로드
배치 40 완료: 200개 문서 업로드
배치 41 완료: 200개 문서 업로드
배치 42 완료: 200개 문서 업로드
배치 43 완료: 200개 문서 업로드
배치 44 완료: 200개 문서 업로드
배치 45 완료: 200개 문서 업로드
배치 46 완료: 200개 문서 업